In [1]:
import pandas as pd
import geopandas as gpd

In [50]:
lodes = pd.read_csv('/home/rishav/Programs/move_od/move_OD/Hamilton_Tennessee_2024-05-29_2024-05-29/county_lodes.csv')
cbg = gpd.read_file('/home/rishav/Programs/move_od/data/states/Tennessee/tl_2023_47_bg.zip')
clifton  = gpd.read_file('/home/rishav/Programs/move_od/data/Clifton.json')
clifton.set_crs('epsg:4326', inplace=True, allow_override=True)
cbg.set_crs('epsg:4326', inplace=True, allow_override=True)

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,GEOIDFQ,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,093,003400,1,470930034001,1500000US470930034001,Block Group 1,G5030,S,3803762,0,+35.9290269,-083.8721965,"POLYGON ((-83.88622 35.92627, -83.88544 35.926..."
1,47,093,001400,3,470930014003,1500000US470930014003,Block Group 3,G5030,S,317086,0,+35.9759890,-083.9484384,"POLYGON ((-83.95561 35.97165, -83.95525 35.971..."
2,47,157,021746,2,471570217462,1500000US471570217462,Block Group 2,G5030,S,863543,0,+35.0413147,-089.8424075,"POLYGON ((-89.84755 35.03527, -89.84745 35.036..."
3,47,173,040102,1,471730401021,1500000US471730401021,Block Group 1,G5030,S,52083268,3244698,+36.3228442,-083.7029007,"POLYGON ((-83.76146 36.30071, -83.76145 36.300..."
4,47,173,040102,2,471730401022,1500000US471730401022,Block Group 2,G5030,S,33795998,2751907,+36.2882119,-083.7758354,"POLYGON ((-83.80645 36.30936, -83.80420 36.311..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4557,47,079,969002,2,470799690022,1500000US470799690022,Block Group 2,G5030,S,60636695,13711004,+36.3404992,-088.1580318,"POLYGON ((-88.22956 36.35051, -88.22804 36.351..."
4558,47,079,969002,1,470799690021,1500000US470799690021,Block Group 1,G5030,S,97441730,26568,+36.4236615,-088.1957062,"POLYGON ((-88.26962 36.41169, -88.26960 36.411..."
4559,47,079,969700,3,470799697003,1500000US470799697003,Block Group 3,G5030,S,55034595,9014199,+36.2573642,-088.1387067,"POLYGON ((-88.20251 36.26068, -88.20219 36.260..."
4560,47,079,969001,1,470799690011,1500000US470799690011,Block Group 1,G5030,S,54627905,18339564,+36.4631505,-088.0882630,"POLYGON ((-88.16133 36.46093, -88.16131 36.461..."


In [51]:
lodes['h_geocode'] = lodes['h_geocode'].astype(str)
lodes['w_geocode'] = lodes['w_geocode'].astype(str)
lodes['h_geocode'] = lodes['h_geocode'].str[:-3]
lodes['w_geocode'] = lodes['w_geocode'].str[:-3]

In [52]:
cbg_clifton = gpd.sjoin(cbg, clifton, how='inner')

# Extract the unique geocodes within Clifton
clifton_geocodes = cbg_clifton['GEOID'].unique()

# Check which rows have both home and work geocodes within the Clifton region
lodes['in_region'] = lodes.apply(
    lambda row: row['h_geocode'] in clifton_geocodes and row['w_geocode'] in clifton_geocodes,
    axis=1
)

# Calculate the total jobs in-region and out-region
total_jobs_in_region = lodes[lodes['in_region']]['total_jobs'].sum()
total_jobs_out_region = lodes[~lodes['in_region']]['total_jobs'].sum()

# Calculate the ratio
if total_jobs_out_region > 0:
    ratio = total_jobs_in_region / total_jobs_out_region
else:
    ratio = float('inf')  # Handle case where there are no out-region jobs


In [ ]:
lodes['total_jobs'].sum()

109314

In [53]:
print(f'Total jobs in-region: {total_jobs_in_region}')
print(f'Total jobs out-region: {total_jobs_out_region}')
print(f'Ratio of in-region to out-region jobs: {ratio}')

Total jobs in-region: 900
Total jobs out-region: 108414
Ratio of in-region to out-region jobs: 0.008301510874979247
